In [1]:
import pickle
import json
import sys
import os
import string
import requests
import ast  # for string to list: ast.literal_eval()
from bs4 import BeautifulSoup
import bs4
import time
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import warnings

In [2]:
data_dir = "/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/2020/"
files = os.listdir(data_dir)
files.sort()
files.remove("covid_monthly_444origo.csv")
data_files = [data_dir + file for file in files]

In [27]:
df = pd.read_csv("/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/2020/all_site_2020.csv")

In [28]:
df["month"]=df["date"].str.split("-").apply(lambda x: x[1])
df["day"]=df["date"].str.split("-").apply(lambda x: x[2])

In [29]:
for i in tqdm(range(df.shape[0])):
    if len(df.loc[i,"month"]) == 1:
        df.loc[i,"month"] = "0" + df.loc[i,"month"]
    if len(df.loc[i,"day"]) == 1:
        df.loc[i,"day"] = "0" + df.loc[i,"day"]

100%|██████████| 224293/224293 [21:37<00:00, 172.81it/s]


In [30]:
df["year"]=df["year"].apply(str)

In [31]:
df["date"] = df.agg('{0[year]}-{0[month]}-{0[day]}'.format, axis=1)

In [32]:
df["date"].apply(len).value_counts()

10    224293
Name: date, dtype: int64

In [33]:
df.to_csv("/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/2020/all_site_2020.csv")

In [44]:
for file in data_files:
    df = pd.read_csv(file,index_col=0)
    print(df.columns)

Index(['link', 'rovat', 'date', 'year', 'month', 'page', 'Title', 'Text',
       'Author', 'Tags'],
      dtype='object')
Index(['link', 'year', 'date', 'month', 'page', 'Title', 'Text', 'links',
       'Author', 'Category', 'Tags'],
      dtype='object')
Index(['link', 'rovat', 'Title', 'Text', 'Author', 'date', 'Tags'], dtype='object')
Index(['datum', '_mod_ts', '__properties', 'rovat', 'rovat_url', 'rovat_slug',
       'kiadvany', 'kiadvany_url', 'tags', 'actual_fb_like', 'ts', 'kep',
       'cim', 'ajanlo', 'alcim', 'url', 'korhatar_stamp', 'alnevek', 'lead',
       'month', 'page', 'Text', 'Author', 'Tags'],
      dtype='object')
Index(['link', 'rovat', 'felosztas', 'Title', 'links', 'Author', 'Text',
       'year', 'month', 'day', 'Tags', 'Source'],
      dtype='object')
Index(['link', 'date', 'month', 'rovat', 'page', 'Title', 'Text', 'links',
       'Author', 'Tags'],
      dtype='object')


In [64]:
_24hu = pd.read_csv(data_files[0],index_col=0)[["link","rovat","date","page","Title","Text","Author","Tags"]].rename(columns={"rovat":"category"})
_24hu.columns = [item.lower() for item in _24hu.columns]

In [164]:
_444 = pd.read_csv(data_files[1],index_col=0)[["link","Category","date","page","Title","Text","Author","Tags"]]
_444.columns = [item.lower() for item in _444.columns]

In [77]:
_888 = pd.read_csv(data_files[2],index_col=0)[["link","rovat","date","Title","Text","Author","Tags"]].rename(columns={"rovat":"category"})


_888["page"] = "888"
_888.columns = [item.lower() for item in _888.columns]

In [152]:
index = pd.read_csv(data_files[3],index_col=0).drop(columns=["tags"]).rename(columns={"rovat":"category","url":"link","_mod_ts":"date","cim":"title"}).reset_index(drop=True)
index.columns = [item.lower() for item in index.columns]

index = index[["link","date","category","title","author","text","tags","page"]]

index["date"] = index["date"].str.split(" ").apply(lambda x: x[0])

In [192]:
mno = pd.read_csv(data_files[4],index_col = 0)
mno.columns = [item.lower() for item in mno.columns]
mno = mno.rename(columns={"rovat":"category"}).dropna(subset=["text","year"]).reset_index(drop=True)

mno["year"] = mno["year"].apply(int).apply(str)
mno["month"] = mno["month"].apply(int).apply(str)
mno["day"] = mno["day"].apply(int).apply(str)

for i in tqdm(range(mno.shape[0])):
    mno.loc[i,"date"] = "-".join([mno.loc[i,"year"],mno.loc[i,"month"],mno.loc[i,"day"]]) 

100%|██████████| 45157/45157 [03:53<00:00, 193.55it/s]


In [193]:
mno = mno[["link","date","category","title","author","text","tags"]]
mno["page"] = "mno"

In [194]:
origo = pd.read_csv(data_files[5],index_col = 0).rename(columns={"rovat":"category"})
origo.columns = [item.lower() for item in origo.columns]
origo = origo[["link","date","category","title","author","text","tags","page"]]
origo["date"] = origo["date"].apply(lambda x: str(x)[0:4] + "-" + str(x)[4:6] +"-"+ str(x)[6:])

In [195]:
all_sites = pd.concat([_24hu,index,origo,_444,_888,mno])
all_sites["page"]=all_sites["page"].apply(str)

In [196]:
all_sites.isnull().sum()

link            0
category        0
date            0
page            0
title       15339
text        26094
author      33743
tags        29091
dtype: int64

In [199]:
all_sites["year"] = all_sites["date"].str.split("-").apply(lambda x: x[0])

In [205]:
all_sites = all_sites.loc[all_sites["year"] == "2020"].reset_index(drop=True)

In [209]:
all_sites.to_csv(data_dir + "all_site_2020.csv")

In [208]:
data_dir

'/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/2020/'

külföld          3993
politika         1965
egészségügy      1482
bűnügy           1023
járvány           853
                 ... 
kereszténység       1
tömeg               1
demográfia          1
media               1
mik vannak?!        1
Name: category, Length: 249, dtype: int64

In [52]:
grouped = df.groupby(["Page","Author"])["Link"].count().reset_index().sort_values(by="Link")

In [74]:
grouped.query("(Page == '888')&(Link > 10)").tail(20)

,Page,Author,Link
188,888,Bohár Dániel,11
160,888,888.hu - V4NA,12
193,888,Filipp Dávid,17
148,888,888,23
150,888,888 / MTI,29
220,888,Pozsonyi Ádám,31
217,888,"MTI, 888.hu",31
198,888,Horváth K. József,42
213,888,M. Kovács Róbert,51
167,888,888.hu ; MTI,55


___

In [6]:
class Helper:
    import json

    from bs4 import BeautifulSoup
    import bs4

    import pandas as pd
    import numpy as np

    import re

    import warnings

    warnings.filterwarnings("ignore")

    def __init__(self, df):
        None

    def title_main(self, element, id):
        for i, soup in enumerate(self.soups):
            try:
                df.loc[self.soups.index[i], "Title"] = (
                    BeautifulSoup(soup).find(element, id=id).find("h1").get_text()
                )
            except AttributeError:
                df.loc[self.soups.index[i], "Title"] = None

    def link(self):
        for i, soup in enumerate(self.soups):
            soup_for_links = BeautifulSoup(soup).find_all("a")
            link_list = []
            for item in soup_for_links:
                if type(item.get("href")) == str:
                    link_list.append(item.get("href"))
            df.loc[self.soups.index[i], "Links"] = str(list(set(link_list)))

    def text_main(self, element, class_id):
        for i, soup in enumerate(self.soups):
            try:
                soup_for_text = (
                    BeautifulSoup(soup)
                    .find(element, class_=re.compile(class_id))
                    .find_all("p")
                )
                text_list = []
                for item in soup_for_text:
                    if soup_for_text != "":
                        text_list.append(item.text)
                df.loc[self.soups.index[i], "Text"] = " ".join(text_list)
            except AttributeError:
                df.loc[self.soups.index[i], "Text"] = None

    def author_main(self, element, class_id):
        for i, soup in enumerate(self.soups):
            try:
                df.loc[self.soups.index[i], "Author"] = (
                    BeautifulSoup(soup)
                    .find(element, class_=class_id)
                    .get_text()
                    .strip()
                )
            except:
                df.loc[self.soups.index[i], "Author"] = None

    def clean(self):
        self.title()
        self.text()
        self.link()
        self.author()

mainekre returnt tenni

_444
Mandiner
Origo
_24
Ripost
_888
Figyelo
VilagGazdasag
Napi
Alfahir
Index
HVG

In [7]:
class _444(Helper):
    def __init__(self, df):
        page = "444"
        self.data = df.loc[df["Page"] == page]
        self.soups = self.data["Soup"].dropna()

    def title(self):
        Helper.title_main(self, "div", "headline")

    def link(self):
        for i, soup in enumerate(self.soups):
            if len(BeautifulSoup(soup).find_all("article")) != 0:
                soup_for_links = (
                    BeautifulSoup(soup).find_all("article")[0].find_all("a")
                )
                link_list = []
                for item in soup_for_links:
                    if type(item.get("href")) == str:
                        link_list.append(item.get("href"))
                df.loc[self.soups.index[i], "Links"] = str(list(set(link_list)))
            else:
                df.loc[self.soups.index[i], "Links"] = []

    def text(self):
        Helper.text_main(self, "main", "col-xs-12 col-md-8 col-lg-7")

    def author(self):
        Helper.author_main(self, "span", "byline__authors")

In [8]:
class Mandiner(Helper):
    def __init__(self, df):
        page = "Mandiner"
        self.data = df.loc[df["Page"] == page]
        self.soups = self.data["Soup"].dropna()

    def title(self):
        Helper.title_main(self, "div", "article")

    def text(self):
        Helper.text_main(self, "div", "articletext")

    def author(self):
        for i, soup in enumerate(self.soups):
            try:
                df.loc[self.soups.index[i], "Author"] = (
                    BeautifulSoup(soup)
                    .find("span", style="text-transform: uppercase; font-weight: 700")
                    .get_text()
                    .strip()
                )
            except AttributeError:
                df.loc[self.soups.index[i], "Author"] = None

In [9]:
class Origo(Helper):
    def __init__(self, df):
        page = "Origo"
        self.data = df.loc[df["Page"] == page]
        self.soups = self.data["Soup"].dropna()

    def title(self):
        Helper.title_main(self, "header", "article-head")

    def text(self):
        for i, soup in enumerate(self.soups):
            try:
                soup_for_text = (
                    BeautifulSoup(soup).find("div", id="article-text").find_all("p")
                )
                text_list = []
                for item in soup_for_text:
                    if soup_for_text != "":
                        text_list.append(item.text)
                df.loc[self.soups.index[i], "Text"] = " ".join(text_list)
            except AttributeError:
                df.loc[self.soups.index[i], "Text"] = None

    def author(self):
        Helper.author_main(self, "span", "article-author")

In [10]:
class _24(Helper):
    def __init__(self, df):
        page = "24.hu"
        self.data = df.loc[df["Page"] == page]
        self.soups = self.data["Soup"].dropna()

    def title(self):
        Helper.title_main(self, "div", "content")

    def text(self):
        Helper.text_main(self, "div", "o-post__body o-postCnt post-body")

    def author(self):
        Helper.author_main(self, "div", "m-author__authorLinkWrap")

In [11]:
class Ripost(Helper):
    def __init__(self, df):
        page = "Ripost"
        self.data = df.loc[df["Page"] == page]
        self.soups = self.data["Soup"].dropna()

    def title(self):
        Helper.title_main(self, "div", "contentholderall")

    def text(self):
        Helper.text_main(self, "div", "content-holder")

    def author(self):
        for i, soup in enumerate(self.soups):
            try:
                df.loc[self.soups.index[i], "Author"] = (
                    BeautifulSoup(soup)
                    .find("a", style="text-decoration:none;")
                    .get_text()
                    .strip()
                )
            except AttributeError:
                df.loc[self.soups.index[i], "Author"] = None

In [12]:
class _888(Helper):
    def __init__(self, df):
        page = "888"
        self.data = df.loc[df["Page"] == page]
        self.soups = self.data["Soup"].dropna()

    def title(self):
        Helper.title_main(self, "div", "cikkholder")

    def text(self):
        Helper.text_main(self, "div", "maincontent8")

    def author(self):
        Helper.author_main(self, "div", "note-block")

In [13]:
class Figyelo(Helper):
    def __init__(self, df):
        page = "Figyelő"
        self.data = df.loc[df["Page"] == page]
        self.soups = self.data["Soup"].dropna()

    def title(self):
        Helper.title_main(self, "div", "article-title")

    def text(self):
        Helper.text_main(self, "div", "entry-content clearfix")

    def author(self):
        Helper.author_main(self, "span", "news__author")

In [14]:
class VilagGazdasag(Helper):
    def __init__(self, df):
        page = "Világgazdaság"
        self.data = df.loc[df["Page"] == page]
        self.soups = self.data["Soup"].dropna()

    def title(self):
        Helper.title_main(self, "div", "content-area")

    def text(self):
        Helper.text_main(self, "div", "entry-content clearfix")

    def author(self):
        Helper.author_main(self, "span", "enews-article-author")

In [15]:
class Napi(Helper):
    def __init__(self, df):
        page = "Napi.hu"
        self.data = df.loc[df["Page"] == page]
        self.soups = self.data["Soup"].dropna()

    def title(self):
        for i, soup in enumerate(self.soups):
            try:
                df.loc[self.soups.index[i], "Title"] = (
                    BeautifulSoup(soup)
                    .find("div", class_="article")
                    .find("h1")
                    .get_text()
                )
            except AttributeError:
                df.loc[self.soups.index[i], "Title"] = None

    def text(self):
        Helper.text_main(self, "div", "article")

    def author(self):
        Helper.author_main(self, "a", "GAHitCounter bl")

In [16]:
class Alfahir(Helper):
    def __init__(self, df):
        page = "Alfahír"
        self.data = df.loc[df["Page"] == page]
        self.soups = self.data["Soup"].dropna()

    def title(self):
        for i, soup in enumerate(self.soups):
            try:
                df.loc[self.soups.index[i], "Title"] = (
                    BeautifulSoup(soup).find("h1", class_="page-title").get_text()
                )
            except AttributeError:
                df.loc[self.soups.index[i], "Title"] = None

    def text(self):
        Helper.text_main(self, "div", "article-content")

    def author(self):
        Helper.author_main(
            self,
            "div",
            "field field--name-username field--type-ds field--label-hidden field--item",
        )

In [17]:
class Index(Helper):
    def __init__(self, df):
        page = "Index"
        self.data = df.loc[df["Page"] == page]
        self.soups = self.data["Soup"].dropna()

    def title(self):
        Helper.title_main(self, "div", "content")

    def text(self):
        Helper.text_main(self, "div", "cikk-torzs")

    def author(self):
        Helper.author_main(self, "div", "szerzok_container")

In [18]:
class HVG(Helper):
    def __init__(self, df):
        page = "HVG"
        self.data = df.loc[df["Page"] == page]
        self.soups = self.data["Soup"].dropna()

    def title(self):
        Helper.title_main(self, "div", "perspective")

    def text(self):
        Helper.text_main(self, "div", "article-menu_main")

    def author(self):
        Helper.author_main(self, "div", "author-name")